# Introducing XGA Products

In this tutorial I will briefly touch on XGA's product classes, which are a key part of the internal makeup of the module. They act as an interface between XGA's functions and the data in different types of X-ray data (images, exposure maps, spectra etc.), and contain many useful and convenient methods for analysis and the extraction of information.

In [3]:
from astropy.units import Quantity

from xga.sources import GalaxyCluster

## What are products?

Most XGA products are what we use to wrap various types of data produced by XMM's SAS routines, as well as providing various product specific functionality. Generally, the user is unlikely to ever define a product instance themselves, internal methods in a source object and the SAS wrapper functions will define, check, and store the products. Those products that are generated by XGA's SAS interface have an added ability to parse the stderr output of the SAS routines used to generate them, then flag any recognised errors (by comparing to an archive of known SAS errors).

There are some XGA products that are not wrappers for SAS generated data products, but are purely for storing and providing access to XGA generated data. These are not based on the same superclass as the other products, but are stored in the same way.

Products instantiated by XGA are immediately 'associated' with a source object, though beyond storing the name of the source object in the product's internal structure, they have no knowledge or awareness of the source and its properties. This was by design, to make sure that any functionality built into a product would work regardless of the type of source, and even if it was defined independently of any source at all.

## What types of product are there?

* **BaseProduct** - The superclass for many of the standard XGA product classes, there isn't really any reason for a BaseProduct to be declared.

    * **EventList** - A very simple product class which differs only slightly from the BaseProduct class, its only used to store path and header information for the XMM event lists that the configuration file points XGA at.

    * **Image** - An extremely useful product with many extra features, it is used to wrap fits images. The data and header information are read into memory (when required), and can be accessed with properties and attributes of an Image instance. A view method (to look at the image and overlay extra information), and a coordinate transformation method are examples of the built-in functionality.

        * **ExpMap** - A subclass of the XGA Image class, this is a very simple extension to Image that adds a method to easily retrieve an exposure time at a given angular or pixel coordinate.

        * **RateMap** - The class that most photometric analyses are based around, also a subclass of Image. Instantiating this class requires the user pass matching Image and ExpMap instances (same ObsID, instrument etc.) A count rate map is then calculated from this information. This has several added methods, including the ability to retrieve a count rate at a given coordinate, and different peak finding methods.

        * **PSF** - This wraps two-dimensional PSF images generated by a routine such as [psfgen](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/psfgen/index.html), and can be used to PSF correct images and ratemaps. Few methods have been added, and it is unlikely a user will ever need to interact directly with this. The added functionality here is the ability to resample the PSF at a scale provided by a passed in Image object.
    
    * **Spectrum** - A complicated sub-class of BaseProduct, it wraps and stores a spectrum generated by a SAS routine, including storing paths to all of the other files necessary to analyse it (e.g. RMF, ARF, background spectrum). When a source that has had spectra associated with it is passed to an XGA XSPEC function, the fit results are added to the spectrum objects. This has methods to retrieve fit results, as well as view the fitted Spectrum.



* **BaseAggregateProduct** - Another base product class, this one is designed to store a group of related XGA products.
    * **PSFGrid** - An XGA object used during the PSF correction of images and ratemaps, it stores a grid of PSF objects generated at different spatial positions on the XMM detectors. It provides access to those PSFs on request from the PSF correction function, and context as to which PSF was generated at which position.
    
    * **AnnularSpectra** - This holds sets of XGA Spectrum products generated in concentric annuli, **but is not yet fully implemented**.



* **BaseProfile1D** - Here we move to the special products that don't wrap existing X-ray data products, these are entirely generated by XGA
    * **SurfaceBrightness1D** -
    
    * **GasDensity1D** - 
    
    * **GasMass1D** - 


* **BaseAggregateProfile1D** - 


* **ScalingRelation** - 


* **AggregateScalingRelation** - 


## Retrieving products from a source